<a href="https://colab.research.google.com/github/alsh626626/CE888/blob/main/Assignment/Assignment1_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>